In [1]:
import pandas as pd
import numpy as np
import random
import os
import random
import shutil

train 1210
test 302

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
import shutil
import time
import pandas as pd
import random

random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

C:\Users\msba7028gpu\anaconda3\envs\d2l\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\msba7028gpu\anaconda3\envs\d2l\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
random.seed(100)
new_data_dir = r"C:\Users\msba7028gpu\Desktop\train_valid_test"  

# image augmentation

In [4]:
random.seed(100)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0),  
                                 ratio=(3.0/4.0, 4.0/3.0)),
    transforms.RandomHorizontalFlip(),
    # randomly change brightness, contrast, saturation and hue
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.5),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])


transform_test = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# data loading

In [5]:
train_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'train'),
                                            transform=transform_train)
test_ds = torchvision.datasets.ImageFolder(root=os.path.join(new_data_dir, 'test'),
                                            transform=transform_test)
batch_size = 32   #batch size can change
train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)  # shuffle=False

# k-fold

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.CrossEntropyLoss()

#dataset = ConcatDataset([train_ds, test_ds])
dataset = train_ds

num_epochs=60
batch_size=32
#k=5
#splits=KFold(n_splits=k,shuffle=True,random_state=42)
#foldperf={}

# Define Model

In [7]:
def get_net():
    net = torchvision.models.resnet18(pretrained=True)
    net.fc = nn.Linear(net.fc.in_features, 8)
    nn.init.xavier_uniform_(net.fc.weight)
    return net

# CV

In [8]:
def train_batch_ch13(net, X, y, loss, trainer, devices):
    """Train for a minibatch with mutiple GPUs (defined in Chapter 13)."""
    if isinstance(X, list):
        
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    net.train()
    trainer.zero_grad()
    pred = net(X)
    l = loss(pred, y)
    l.sum().backward()
    trainer.step()
    train_loss_sum = l.sum()
    train_acc_sum = d2l.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

In [9]:
from d2l import torch as d2l
def train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus()):
    """Train a model with mutiple GPUs (defined in Chapter 13)."""
    timer, num_batches = d2l.Timer(), len(train_iter)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples,
        # no. of predictions
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = train_batch_ch13(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
    
    loss = metric[0] / metric[2]
    train_acc = metric[1] / metric[3]

    return loss,train_acc,test_acc

In [10]:
def train_fine_tuning(net, learning_rate, trainloader, testloader, batch_size, num_epochs,
                      param_group=True):
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': learning_rate * 10}],
                                lr=learning_rate, weight_decay=0.01)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.01)
    
    loss,train_acc,test_acc = train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus())
    return loss,train_acc,test_acc

In [11]:
def cv_result(learning_rate,batch_size,num_epochs,param_group=True):
    loss_ls =[]
    train_acc_ls =[]
    test_acc_ls = []
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

        print('Fold {}'.format(fold + 1))

        #all test there means validation

        train_sampler = SubsetRandomSampler(train_idx) 
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
        
        net = get_net()

        loss,train_acc,test_acc = train_fine_tuning(net, learning_rate, train_loader, test_loader, batch_size, num_epochs,
                          param_group=True)

        loss_ls.append(loss)
        train_acc_ls.append(train_acc)
        test_acc_ls.append(test_acc)
        print('loss: ',loss)
        print('train: ',train_acc)
        print('test: ',test_acc)
    return loss_ls,train_acc_ls,test_acc_ls

In [13]:
loss_ls,train_acc_ls,test_acc_ls = cv_result(learning_rate = 5e-5,batch_size=32,num_epochs = 25)

Fold 1
loss:  0.6228450002749104
train:  0.7834710743801653
test:  0.7417218543046358
Fold 2
loss:  0.5865348815917969
train:  0.8041322314049587
test:  0.7814569536423841
Fold 3
loss:  0.5925184155298658
train:  0.8066115702479338
test:  0.7615894039735099
Fold 4
loss:  0.6166397591267736
train:  0.7776859504132232
test:  0.7880794701986755
Fold 5
loss:  0.5718248004755698
train:  0.8049586776859504
test:  0.7947019867549668


In [14]:
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

5fold_average_loss: 0.598, 5fold_average_train_accuracy: 0.795, 5fold_average_test_accuracy: 0.774


In [15]:
loss_ls,train_acc_ls,test_acc_ls = cv_result(learning_rate = 10e-5,batch_size=32,num_epochs = 35)

Fold 1
loss:  0.3773840387990652
train:  0.8785123966942149
test:  0.7947019867549668
Fold 2
loss:  0.38814979131556737
train:  0.868595041322314
test:  0.8145695364238411
Fold 3
loss:  0.3973514454423889
train:  0.8677685950413223
test:  0.7947019867549668
Fold 4
loss:  0.38299530951444766
train:  0.8743801652892562
test:  0.804635761589404
Fold 5
loss:  0.3695365759952009
train:  0.8793388429752066
test:  0.7980132450331126


In [16]:
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

5fold_average_loss: 0.383, 5fold_average_train_accuracy: 0.874, 5fold_average_test_accuracy: 0.801


In [17]:
def train_fine_tuning(net, learning_rate, trainloader, testloader, batch_size, num_epochs,
                      param_group=True):
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': learning_rate * 10}],
                                lr=learning_rate, weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.001)
    
    loss,train_acc,test_acc = train_model(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus())
    return loss,train_acc,test_acc

In [18]:
loss_ls,train_acc_ls,test_acc_ls = cv_result(learning_rate = 10e-5,batch_size=32,num_epochs = 25)

Fold 1
loss:  0.4386138025394156
train:  0.8504132231404958
test:  0.7913907284768212
Fold 2
loss:  0.5058183094686713
train:  0.8264462809917356
test:  0.7847682119205298
Fold 3
loss:  0.4878374127317066
train:  0.8404958677685951
test:  0.8211920529801324
Fold 4
loss:  0.43650174298562294
train:  0.8487603305785124
test:  0.8211920529801324
Fold 5
loss:  0.48207707129234123
train:  0.8314049586776859
test:  0.7880794701986755


In [19]:
loss_avg = sum(loss_ls)/len(loss_ls)
train_acc_avg = sum(train_acc_ls)/len(train_acc_ls)
test_acc_avg = sum(test_acc_ls)/len(test_acc_ls)
print(f'5fold_average_loss: {loss_avg:.3f}, 5fold_average_train_accuracy: '
      f'{train_acc_avg:.3f}, 5fold_average_test_accuracy: {test_acc_avg:.3f}')

5fold_average_loss: 0.470, 5fold_average_train_accuracy: 0.840, 5fold_average_test_accuracy: 0.801
